In [1]:
from python_graphql_client import GraphqlClient
import json
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
endpoint = 'https://api2.mapofzones.com/v1/graphql'

# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint=endpoint)

# Create the query string and variables required for the request.
query = """
    query ZoneName($zone: String!) {
        blockchain: flat_blockchains(where: {network_id: {_eq: $zone}}) {
            name
            __typename
        }
    }
"""

variables = {"zone":"stride-1"}

# Synchronous request
data = client.execute(query=query, variables=variables)
print(data) 

{'data': {'blockchain': [{'name': 'Stride', '__typename': 'flat_blockchains'}]}}


In [34]:
query = """
query AseetsTable {
  assets: flat_tokens {
    blockchain: blockchainByBlockchain {
      name
      __typename
    }
    symbol
    logoUrl: logo_url
    price
    price24hDiffPercent: price_day_diff_percent
    price7dDiffPercent: price_week_diff_percent
    marketCap: market_cap
    volume24h: token_day_trading_volume
    volume24hDiffPercent: token_day_trading_volume_diff_percent
    onChainSupply: on_chain_supply
    priceChart: token_charts(
      where: {chart_type: {_eq: "price_weekly"}}
      order_by: {point_index: asc}
    ) {
      price: point_value
      __typename
    }
    __typename
  }
}
"""

variables = {}

data = client.execute(query=query, variables=variables)
with open('data/marketcap.json', 'w') as f:
    json.dump(data, f)

In [32]:
query = """
query ZonesMap($period: Int!, $isMainnet: Boolean!) {
  zonesStats: flat_blockchains(where: {is_mainnet: {_eq: $isMainnet}}) {
    ...ZoneBaseInfoV2
    isMainnet: is_mainnet
    switchedStats: blockchain_switched_stats(
      where: {timeframe: {_eq: $period}, is_mainnet: {_eq: $isMainnet}}
    ) {
      ibcVolume: ibc_cashflow
      ibcVolumeIn: ibc_cashflow_in
      ibcVolumeOut: ibc_cashflow_out
      ibcVolumeRating: ibc_cashflow_rating
      ibcTransfersRating: ibc_transfers_rating
      dauRating: active_addresses_cnt_rating
      totalTxsRating: txs_rating
      __typename
    }
    __typename
  }
  zonesGraphs: flat_blockchain_relations(
    where: {blockchain: {is_mainnet: {_eq: $isMainnet}}, blockchainByBlockchainSource: {is_mainnet: {_eq: $isMainnet}}, timeframe: {_eq: $period}}
  ) {
    source: blockchain_source
    target: blockchain_target
    ibcVolume: ibc_cashflow
    __typename
  }
}

fragment ZoneBaseInfoV2 on flat_blockchains {
  zone: network_id
  logoUrl: logo_url
  name: name
  __typename
}
"""

variables = {"period": 24, "isMainnet": True}
data = client.execute(query=query, variables=variables)

with open('data/zonemap.json', 'w') as f:
    json.dump(data, f)

In [18]:
mapdata = dict()

with open('data/zonemap.json', 'r') as f:
    mapdata = json.load(f)

zones = mapdata['data']['zonesStats']
channels = mapdata['data']['zonesGraphs']

In [50]:
marketdata_raw = dict()
with open('data/marketcap.json', 'r') as f:
    marketdata_raw = json.load(f)

marketdata = dict()
for asset in marketdata_raw['data']['assets']:
    marketdata[asset['blockchain']['name']] = asset['marketCap']

marketdata

{'Agoric': 401990795.619125,
 'Akash': 52244643.4735865,
 'Axelar': 3425700.09859555,
 'Bitcanna': 7727829.48730632,
 'BitSong': 1078320.7474602,
 'Bostrom': 12705070.0792921,
 'Canto': 133792759.814222,
 'cheqd': 29455587.6018665,
 'Chihuahua': 8750496.74738753,
 'Terra': None,
 'Comdex': 13825422.971917,
 'Persistence': 94523878.0819665,
 'Crescent Network': 2102419.70522711,
 'Cronos': 183378257.698603,
 'Crypto.org': 1940840362.95697,
 'Konstellation': 2196952.2347101,
 'Desmos': 4446458.34286593,
 'Dig Chain': 394824.2004003,
 'E-money': 3581914.09558784,
 'Evmos': 310015103.185121,
 'OKX Chain': 74.6459998498542,
 'Fetch.ai': 262173641.897068,
 'Gravity Bridge': 21326862.0022816,
 'Ixo': 2541181.69978464,
 'Injective': 92166612.253146,
 'Starname': 627943.204641248,
 'Iris': 31407421.2462078,
 'Jackal': 30024787.5246271,
 'Juno': 125944370.422194,
 'Kujira': 73012094.2289174,
 'KiChain': 60488704.8870771,
 'Band': 254671395.461194,
 'LikeCoin': 2478664.63906548,
 'Cosmos Hub': 44

In [57]:


G = nx.Graph()
for zone in zones:
    if zone['name'] in marketdata and marketdata[zone['name']] != None:
        market_cap = int(marketdata[zone['name']])
        G.add_node(zone['zone'], weight=market_cap)

for chan in channels:
    if chan["ibcVolume"] == 0:
        continue
    if chan["source"] not in G.nodes:
        continue
    if chan['target'] not in G.nodes:
        continue
    G.add_edge(chan["source"], chan["target"], weight=chan["ibcVolume"])

# G.nodes

# max weight clique in terms of node weights
nx.max_weight_clique(G)



(['evmos_9001-2',
  'kaiyo-1',
  'axelar-dojo-1',
  'juno-1',
  'cosmoshub-4',
  'osmosis-1'],
 4938452320)